In [1]:
import unidecode
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from datetime import timedelta

In [2]:
from matplotlib import pyplot as plt

In [3]:
from configs import lista_nombres_nuevos, dict_nombres_nuevos, lista_columnas_necesarias, barrios

Adaptaciones para la tabla (repaso)

(idea para el futuro: detectar nombres de vegetales en nombres genéricos)
Quitar columnas no frutas y verduras
Cambiar nombres
Convertir fechas
crear fechas de entrega


Agrupar qué?

IDEA: armar un deslizador para graficar cantidad de vendidos para distintos tipos de frutas (deslizar cambia la fruta)

In [4]:
def crear_diccionario_nombres_nuevos(df):
    
    nombres_nuevos = {}
    
    lista_nombres_viejos = list(df.columns)
    for nombre_viejo in lista_nombres_viejos:
        
        # limpiamos columnas del df (lowercase, acentos y emojis)
        nombre_viejo_limpio = unidecode.unidecode(nombre_viejo.lower())
        
        # si tiene combo lo deja igual
        if ('combo' in nombre_viejo_limpio) or ('bolson' in nombre_viejo_limpio):
            for key, value in dict_nombres_nuevos.items():
                if key in nombre_viejo_limpio:
                    nombres_nuevos[nombre_viejo] = value
        else:
            for nombre_nuevo in lista_nombres_nuevos:
                # si encuentra un substring en nombres nuevos que matchee se lo asigna
                if nombre_nuevo in nombre_viejo_limpio and not 'budin' in nombre_viejo_limpio:
                    nombres_nuevos[nombre_viejo] = nombre_nuevo
            
    return nombres_nuevos 

In [7]:
def sabado_posterior(fecha):
    sabado = fecha
    while(sabado.weekday() != 5):
        sabado = sabado + timedelta(days = 1)
    return sabado

cortar_fecha = lambda fecha_hora: fecha_hora.split(' ')[0]

def agregar_fecha_entrega(df):
    df['marca temporal'] = df['marca temporal'].apply(cortar_fecha)
    df['marca temporal'] = pd.to_datetime(df['marca temporal'],format='%Y/%m/%d')
    df['fechas entrega'] = df['marca temporal'].apply(sabado_posterior)
    return df

In [8]:
def limpiar_barrios(retirar):
    
    for barrio in barrios:
        if barrio in str(retirar):
            return barrio
    
    return retirar

In [9]:
def crear_tabla_base(df):
    # rellena nans con 0
    df_filtrado = df.copy().fillna(0)
    
    # cambia los nombres de las columnas
    df_filtrado.rename(columns=crear_diccionario_nombres_nuevos(df), inplace=True)

    # filtra columnas con las que trabajar
    df_filtrado = df_filtrado[lista_columnas_necesarias]
    
    # agrega fecha de entrega segun fecha de pedido
    df_filtrado = agregar_fecha_entrega(df_filtrado)
    
    # limpia columna retirar
    df_filtrado['retirar'] = df_filtrado['retirar'].apply(limpiar_barrios)
    return df_filtrado 

In [10]:
df = pd.read_csv('2022_anonimize.csv')

df_filtrado = crear_tabla_base(df)
df_filtrado.head()

,marca temporal,nombre de usuario,choclo,palta,tomate perita,morron verde,uva,berenjena agroecologica,ajo,ciruela,manzana roja,limon,pera,naranja,naranja,durazno,morron rojo,zapallito redondo,tomate cherry,mandarina,frutillas,banana,mango,arandanos,arandanos,jengibre,jengibre,retirar,bolson de verdes,bolson de pesadas,combo bolson verdes pesadas,bolson citricos,fechas entrega
0,2021-03-28,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Palermo,1.0,1.0,0.0,0.0,2021-04-03
1,2021-03-28,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Palermo,1.0,1.0,0.0,0.0,2021-04-03
2,2021-03-28,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Urquiza,1.0,1.0,0.0,0.0,2021-04-03
3,2021-03-28,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Palermo,2.0,1.0,0.0,0.0,2021-04-03
4,2021-03-28,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Palermo,1.0,1.0,0.0,0.0,2021-04-03


In [26]:
#seleccionar_rango_fechas = lambda df,fecha_inicial,fecha_final: df.loc[fecha_final <= df['fechas entrega']].reset_index() 
def seleccionar_rango_fechas_entrega(df,fecha_inicial,fecha_final):
    df_seleccionado = df.loc[fecha_inicial <= df['fechas entrega']].reset_index()
    df_seleccionado = df_seleccionado.loc[fecha_final >= df['fechas entrega']].reset_index()
    return df_seleccionado

seleccionar_puntos_entrega = lambda df,lista_barrios: df.loc[df['retirar'].isin(lista_barrios)].reset_index()

ranking_pedidos_por_usuario = lambda df: df.groupby(by=['nombre de usuario']).size().reset_index()

,nombre de usuario,0
0,0,29
1,1,20
2,2,4
3,3,2
4,4,2
...,...,...
362,362,1
363,363,1
364,364,1
365,365,1


In [28]:
def crear_lista_pedidos_unicos(df):
    df_cantidad_de_pedidos_por_usuario = ranking_pedidos_por_usuario(df)
    lista_usuarios = df_cantidad_de_pedidos_por_usuario['nombre de usuario'].unique()
    lista_pedidos_unicos = []
    for usuario in lista_usuarios:
        if df_cantidad_de_pedidos_por_usuario[0].loc[usuario] == 1:
            lista_pedidos_unicos.append(usuario)
    return lista_pedidos_unicos

crear_lista_pedidos_unicos(df_filtrado)

def filtrar_pedidos_unicos(df):
    usuarios_pedido_unico = crear_lista_pedidos_unicos(df)
    df_pedidos_unicos = df.loc[df['nombre de usuario'].isin(usuarios_pedido_unico)].reset_index()
    return df_pedidos_unicos

,index,marca temporal,nombre de usuario,choclo,palta,tomate perita,morron verde,uva,berenjena agroecologica,ajo,ciruela,manzana roja,limon,pera,naranja,naranja,durazno,morron rojo,zapallito redondo,tomate cherry,mandarina,frutillas,banana,mango,arandanos,arandanos,jengibre,jengibre,retirar,bolson de verdes,bolson de pesadas,combo bolson verdes pesadas,bolson citricos,fechas entrega
0,5,2021-03-28,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Urquiza,1.0,1.0,0.0,0.0,2021-04-03
1,6,2021-03-28,6,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Urquiza,1.0,1.0,0.0,0.0,2021-04-03
2,9,2021-03-29,9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Urquiza,1.0,2.0,0.0,0.0,2021-04-03
3,17,2021-03-30,17,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Urquiza,1.0,1.0,0.0,0.0,2021-04-03
4,21,2021-03-31,21,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Palermo,1.0,0.0,0.0,0.0,2021-04-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,1020,2022-03-03,362,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Urquiza,0.0,0.0,1.0,0.0,2022-03-05
209,1023,2022-03-03,363,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Crespo,0.0,0.0,0.0,0.0,2022-03-05
210,1026,2022-03-03,364,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,1.0,2022-03-05
211,1028,2022-03-03,365,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villa Crespo,0.0,0.0,1.0,0.0,2022-03-05


Ya tenemos la tabla filtrada y adaptada para poder crear las tablas nuevas que respondan las preguntas que nos interesan

Planificación de los gráficos para implementar:
Para una sola fecha de entrega:
    Lista de productos, seccionada por lista de casas populares (gráfico de barras)
    Cantidad de pedidos totales por casa popular (gráfico de barras)

Para un rango de fechas de entrega:
    Un solo producto, pedidos seccionados por casas populares (gráfico de barras seccionadas) 
    Cantidad de pedidos totales por casa popular (gráfico de barras seccionadas)
    Lista de productos (gráfico de líneas)
    Cantidad de pedidos únicos por casa popular (scatter de colores)

Pedidos únicos:
    Cantidad de pedidos únicos totales por casa popular (gráfico de torta)
